Player Role Team Credits Points

In [26]:
import pandas as pd
import pulp
import warnings 
warnings.filterwarnings('ignore')

In [27]:
df = pd.read_csv("data/sample_twoTteams.csv")
df.head(24)

,Unnamed: 0,Player,Role,Team,Credits,Points
0,0,Cameron Green,Allrounder,MI,9.0,687.4
1,1,Suryakumar Yadav,Batter,MI,9.5,679.8
2,2,Piyush Chawla,Bowler,MI,8.0,657.5
3,3,Ishan Kishan,Keeper,MI,10.5,458.9
4,4,Tilak Varma,Batter,MI,9.0,427.1
5,5,Jason Behrendorff,Bowler,MI,9.0,373.4
6,6,Akash Madhwal,Bowler,MI,9.0,290.5
7,7,Rohit Sharma,Batter,MI,8.5,284.6
8,8,Tim David,Batter,MI,8.5,210.9
9,9,Chris Jordan,Bowler,MI,10.5,64.2


In [28]:
# feature engineering
df = df.join(pd.get_dummies(df[['Role', 'Team']]))
df['playing_xi'] = 1;
df.head(24)

,Unnamed: 0,Player,Role,Team,Credits,Points,Role_Allrounder,Role_Batter,Role_Bowler,Role_Keeper,Team_CSK,Team_MI,playing_xi
0,0,Cameron Green,Allrounder,MI,9.0,687.4,True,False,False,False,False,True,1
1,1,Suryakumar Yadav,Batter,MI,9.5,679.8,False,True,False,False,False,True,1
2,2,Piyush Chawla,Bowler,MI,8.0,657.5,False,False,True,False,False,True,1
3,3,Ishan Kishan,Keeper,MI,10.5,458.9,False,False,False,True,False,True,1
4,4,Tilak Varma,Batter,MI,9.0,427.1,False,True,False,False,False,True,1
5,5,Jason Behrendorff,Bowler,MI,9.0,373.4,False,False,True,False,False,True,1
6,6,Akash Madhwal,Bowler,MI,9.0,290.5,False,False,True,False,False,True,1
7,7,Rohit Sharma,Batter,MI,8.5,284.6,False,True,False,False,False,True,1
8,8,Tim David,Batter,MI,8.5,210.9,False,True,False,False,False,True,1
9,9,Chris Jordan,Bowler,MI,10.5,64.2,False,False,True,False,False,True,1


In [29]:
# decision variable
pickup_status = pulp.LpVariable.dicts('pickup_status', ((obj) for obj in df.index), cat = 'Binary')

# objective function
mo = pulp.LpProblem('Profit maximisation problem', pulp.LpMaximize)
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Points'] for obj in df.index])

# constraints
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Credits']] for obj in df.index) <= 100

# 11 player
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'playing_xi']] for obj in df.index) == 11

# team contraints
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Team_CSK']] for obj in df.index) >= 4
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Team_CSK']] for obj in df.index) <= 7

mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Team_MI']] for obj in df.index) >= 4
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Team_MI']] for obj in df.index) <= 7

# role contraints
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_Keeper']] for obj in df.index) >= 1
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_Keeper']] for obj in df.index) <= 4

mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_Batter']] for obj in df.index) >= 3
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_Batter']] for obj in df.index) <= 6

mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_Bowler']] for obj in df.index) >= 3
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_Bowler']] for obj in df.index) <= 6

mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_Allrounder']] for obj in df.index) >= 1
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_Allrounder']] for obj in df.index) <= 4

In [30]:
mo.solve()

1

In [31]:
# output as a column in df
df['pickup_status'] = 0

for obj in df.index :
  if pickup_status[obj].varValue :
    df['pickup_status'][obj] = 1

In [32]:
fdf = df[df['pickup_status'] == 1].sort_values(['Role_Batter', 'Role_Keeper', 'Role_Allrounder', 'Role_Bowler'], ascending = False)
fdf.head(11)

,Unnamed: 0,Player,Role,Team,Credits,Points,Role_Allrounder,Role_Batter,Role_Bowler,Role_Keeper,Team_CSK,Team_MI,playing_xi,pickup_status
1,1,Suryakumar Yadav,Batter,MI,9.5,679.8,False,True,False,False,False,True,1,1
4,4,Tilak Varma,Batter,MI,9.0,427.1,False,True,False,False,False,True,1,1
13,1,Ruturaj Gaikwad,Batter,CSK,9.5,651.9,False,True,False,False,True,False,1,1
14,2,Devon Conway,Batter,CSK,8.0,599.9,False,True,False,False,True,False,1,1
16,4,Ajinkya Rahane,Batter,CSK,9.0,399.5,False,True,False,False,True,False,1,1
3,3,Ishan Kishan,Keeper,MI,10.5,458.9,False,False,False,True,False,True,1,1
0,0,Cameron Green,Allrounder,MI,9.0,687.4,True,False,False,False,False,True,1,1
12,0,Ravindra Jadeja,Allrounder,CSK,9.0,718.6,True,False,False,False,True,False,1,1
2,2,Piyush Chawla,Bowler,MI,8.0,657.5,False,False,True,False,False,True,1,1
5,5,Jason Behrendorff,Bowler,MI,9.0,373.4,False,False,True,False,False,True,1,1


In [33]:
print(sum(fdf.Credits))

99.0
